In [2]:
# Load Pretrained Model
# w2v_model = KeyedVectors.load_word2vec_format(
#     'models/pretrained/GoogleNews-vectors-negative300.bin', binary=True)
# w2v_model.save('models/pretrained/word2vec_model.gensim')


In [3]:
import json
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors
from tensorflow.keras.preprocessing.text import Tokenizer

# Load dataset
dev_df = pd.read_csv('TREC_dataset/dev.csv')
train_df = pd.read_csv('TREC_dataset/train.csv')
test_df = pd.read_csv('TREC_dataset/test.csv')

print("Training Size: ", train_df.shape[0])
print("Development Size: ", dev_df.shape[0])
print("Test Size: ", test_df.shape[0])

# Load pre-trained Word2Vec model
model_path = 'models/pretrained/word2vec_model.gensim'
word2vec_model = KeyedVectors.load(model_path)

# Combine text data from train, validation, and test sets
combined_texts = pd.concat([train_df['text'], dev_df['text'], test_df['text']])

# Initialize tokenizer
tokenizer = Tokenizer(filters='')

# Fit tokenizer on combined text data
tokenizer.fit_on_texts(combined_texts)

# Create a reduced embedding matrix
vocab_size = len(tokenizer.word_index) + 1  # +1 for padding token
embedding_dim = word2vec_model.vector_size
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in tokenizer.word_index.items():
    if word in word2vec_model.key_to_index:
        embedding_vector = word2vec_model[word]
        embedding_matrix[i] = embedding_vector

# Save embedding matrix and tokenizer
np.save('models/pretrained/embedding_matrix.npy', embedding_matrix)

# Save tokenizer
tokenizer_json = tokenizer.to_json()
with open('models/pretrained/tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))

2023-11-10 18:15:56.775185: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Training Size:  4952
Development Size:  500
Test Size:  500
